In [0]:
spark.sql("CREATE SCHEMA if not EXISTS dicionarios")
spark.sql("""
CREATE TABLE IF NOT EXISTS dicionarios.dicionario_de_metadados (
  origem STRING,
  nome_tabela_origem STRING,
  nome_tabela_transformada STRING,
  dado_sensivel BOOLEAN
)
""")

In [0]:
%pip install faker

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
import random
from faker import Faker
from datetime import datetime

fake = Faker('pt_BR')

# Listas de valores possíveis
marcas_modelos = {
    "Toyota": ["Corolla", "Hilux", "Yaris"],
    "Honda": ["Civic", "HR-V", "Fit"],
    "Ford": ["Ka", "EcoSport", "Ranger"],
    "Chevrolet": ["Onix", "Tracker", "S10"],
    "Volkswagen": ["Gol", "Polo", "T-Cross"]
}

formas_pagamento = ["À vista", "Financiamento", "Consórcio", "Leasing"]
cores = ["Preto", "Branco", "Prata", "Vermelho", "Azul"]
sexo_opcoes = ["Masculino", "Feminino"]

# Gerando os dados fictícios
dados = []
for _ in range(1000):
    sexo = random.choice(sexo_opcoes)
    nome = fake.first_name_male() if sexo == "Masculino" else fake.first_name_female()
    sobrenome = fake.last_name()
    nome_completo = f"{nome} {sobrenome}"
    
    idade = random.randint(18, 70)
    cpf = fake.ssn()
    email = fake.email()
    telefone = fake.phone_number()
    cidade = fake.city()
    estado = fake.state_abbr()
    
    marca = random.choice(list(marcas_modelos.keys()))
    modelo = random.choice(marcas_modelos[marca])
    ano_fabricacao = random.randint(2015, 2024)
    cor = random.choice(cores)
    valor_venda = round(random.uniform(38000, 250000), 2)
    data_venda = fake.date_between(start_date="-2y", end_date="today")
    forma_pagamento = random.choice(formas_pagamento)
    
    dados.append((nome_completo, idade, cpf, sexo, email, telefone, cidade, estado,
                  marca, modelo, ano_fabricacao, cor, valor_venda, data_venda, forma_pagamento))

# Criando o esquema do DataFrame
esquema = StructType([
    StructField("NOME", StringType(), True),
    StructField("IDADE", IntegerType(), True),
    StructField("CPF", StringType(), True),
    StructField("SEXO", StringType(), True),
    StructField("EMAIL", StringType(), True),
    StructField("TELEFONE", StringType(), True),
    StructField("CIDADE", StringType(), True),
    StructField("ESTADO", StringType(), True),
    StructField("MARCA_VEICULO", StringType(), True),
    StructField("MODELO_VEICULO", StringType(), True),
    StructField("ANO_FABRICACAO", IntegerType(), True),
    StructField("COR", StringType(), True),
    StructField("VALOR_VENDA", FloatType(), True),
    StructField("DATA_VENDA", DateType(), True),
    StructField("FORMA_PAGAMENTO", StringType(), True)
])

# Criando o DataFrame no Spark
df_vendas = spark.createDataFrame(dados, schema=esquema)

# Exibir as primeiras linhas
display(spark.table('dicionarios.dicionario_de_metadados'))
display(df_vendas)

In [0]:
spark.sql("CREATE SCHEMA if not EXISTS bronze;"); spark.sql("CREATE SCHEMA if not EXISTS silver;"); spark.sql("CREATE SCHEMA if not EXISTS gold;"); spark.sql("CREATE SCHEMA if not EXISTS landing;")

In [0]:
# Criando as tabelas no esquema bronze
df_vendas.write.mode("overwrite").saveAsTable("landing.sales_oracle_system")